# Practice web scraping

In this notebook I will practice web scraping skills by getting some data off www.eatwild.com. The end product will be a .csv file that I will save for future use.

First check the robots.txt file for permissions. --> No robots.txt file exists for this website. The default is to allow requests.

Start with getting data about farms in just Washington State. Can come back later and do other states.

In [1]:
from bs4 import BeautifulSoup
import requests
import pandas as pd

In [2]:
url = "http://www.eatwild.com/products/washington.html"
response = requests.get(url)

Check that request was successful.

In [3]:
response.status_code

200

Parse response with Beautiful Soup.

In [4]:
soup = BeautifulSoup(response.content, "html.parser")

Farm sections can be identified when a "strong" or "bold" tag comes at the beginning of a "p class="bodyMargin"". Start by collecting all paragraphs that are of class=bodyMargin.

In [5]:
paragraphs = soup.find_all('p', attrs={'class':'bodyMargin'})
paragraphs

[<p class="bodyMargin"> </p>,
 <p class="bodyMargin"><i><b>The following farms and ranches have certified
             that they meet <a href="http://www.eatwild.com/criteria.html" target="_blank" title="read Eatwild's criteria">Eatwild's
             criteria</a> for producing grassfed meat, eggs and dairy products.
             Contact them directly for additional information or to buy their
             products: <a name="acme"></a></b></i></p>,
 <p class="bodyMargin"><strong>3 Sisters</strong> <strong>Family Farms</strong> <em><a href="#3sisters" target="_self" title="link to listing below">see below</a></em></p>,
 <p class="bodyMargin"><strong>3-Bell Ranch</strong> <em><a href="#3bell" target="_self" title="link to listing below">see below</a></em></p>,
 <p class="bodyMargin"><strong>Acme Farm</strong> raises <strong>grass-fed</strong>,  antibiotic-, chemical-, and grain-free <strong>lamb</strong>. <em>We follow organic practices on our farm.</em></p>,
 <p class="bodyMargin">Our  

Next, find only the paragraphs that start with a strong or bold tag and isolate the bolded farm name

In [6]:
paragraphs[13].tags

In [65]:
farm_list=[]
for pa in paragraphs:
    if pa.find('strong'):
        strong_text = pa.find('strong').text
        farm_list.append(strong_text)
    if pa.find('b'):
        bold_text = pa.find('b').text
        farm_list.append(bold_text)
farm_list

["The following farms and ranches have certified\r\n            that they meet Eatwild's\r\n            criteria for producing grassfed meat, eggs and dairy products.\r\n            Contact them directly for additional information or to buy their\r\n            products: ",
 '3 Sisters',
 '3-Bell Ranch',
 'Acme Farm',
 '',
 'Acres in Zion',
 '',
 'Akyla Farms',
 'Amazing Grace Highlands  ',
 'American Alps Ranch',
 'Aurora Farms',
 'Autumn Hills Woollens and Sheep Co',
 'Baron Farms',
 'Basket Flat Ranch',
 'BCS Livestock',
 'Birkeland Farm',
 "Bob's Beef ",
 'Bone Dry Ridge Farm',
 'beef, lamb, pork,',
 'Botany  Bay Farm',
 'Bray Angus',
 "Card's L Hanging\r\n                      S Ranch Grass Fat Beef",
 'Castle Rock  Ranch',
 'Chinook  Farms',
 'Colvin Ranch',
 'Crown S Ranch',
 'Dirty  Boots Ranch',
 'Douglas Farms',
 'beef',
 'Earth\r\n                      Cycle Farm',
 'Ground beef, milk,',
 'Evergreen Students for Sustainable\r\n                      Animal Agriculture',
 'The

That worked pretty well. There's more information about each farm in the last paragraph of each section. A consistent piece of information is the link to the address of the farm. This is specified as an "a" tag with 'title'='Google map'. Use previous_element to find the string preceding the map link.

In [70]:
map_tags = soup.find_all(title='Google map')
farm_names=[]
farmer_names=[]
count =0
for tag in map_tags:
    count = count+1
    #print(count)
    name_string_temp = tag.previous_element
    farm_name_i = name_string_temp.split(',')[0]
    farm_names.append(farm_name_i)
    if len(name_string_temp.split(','))>=2:
    #if name_string_temp.split(',')[1]:
        farmer_name_i = name_string_temp.split(',')[1].strip()
    else:
        farmer_name_i = ''
    farmer_names.append(farmer_name_i)
farm_names[0]
farmer_names[0]


TypeError: 'NoneType' object is not callable

In [69]:
farmerA = map_tags[48].previous_element
print(farmerA)
if map_tags[16].previous_element.split(',')[1]:
    print("farmer exists")

 On  the Lamb Farm, 


IndexError: list index out of range

In [73]:
h_rows = soup.find_all('hr')
info_ps = []
for i in h_rows:
    info_ps.append(i.previous_sibling('p'))

TypeError: 'NoneType' object is not callable

In [82]:
print(h_rows[4])
print(h_rows[4].find_previous_sibling('p'))

<hr color="#006600" noshade="" size="3" width="95%"/>
<p class="bodyMargin">Acme Farm, Nancy Hibbing, <a href="https://www.google.com/maps?q=5808+Saxon+Road,+Acme+WA+9822&amp;hl=en&amp;ll=47.820532,-122.085571&amp;spn=2.452748,5.300903&amp;sll=37.0625,-95.677068&amp;sspn=46.005754,84.814453&amp;hnear=5808+Saxon+Rd,+Acme,+Whatcom,+Washington+98220&amp;t=m&amp;z=8" target="_blank" title="Google map">5808 Saxon Road, Acme WA</a> 98220. (360) 722-1322. <br/>
      E-mail: <a href="mailto:acmefarm@msn.com" target="_blank" title="email farm">acmefarm@msn.com</a>.<b><a name="acres"></a></b></p>
